In [2]:
from preprocess_dataset import standardised_X as X, dummy_Y as Y

In [3]:
import pandas as pd
import numpy as np

In [4]:
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [5]:
print(X.shape[0])
print(Y.shape)
print(len(X[1]))

9679795
(9679795, 5)
29


In [ ]:
features = len(X[0])
samples = X.shape[0]
train_len = 50
input_len = samples - train_len
input = np.zeros((samples - train_len, train_len, features))

for i in range(input_len):
    temp = np.zeros((train_len, features))
    for j in range(i, i + train_len - 1):
        temp[j-i] = X[j]
    input[i] = temp

In [130]:
output_len = samples - train_len
output_features = Y.shape[1]
output = np.zeros((samples - train_len, train_len, output_features))
output.shape

for i in range(output_len):
    temp = np.zeros((train_len, output_features))
    for j in range(i, i + train_len - 1):
        temp[j-i] = Y[j]
    output[i] = temp

In [131]:
X_train, X_test, Y_train, Y_test = train_test_split(input, Y[50:20000], test_size = 0.2, random_state = 42)

In [132]:
def create_baseline():
    model = Sequential()
    
    model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
#     model.add(Bidirectional(LSTM(64, activation='tanh')))
#     model.add(Bidirectional(LSTM(64, activation='tanh')))
#     model.add(Bidirectional(LSTM(64, activation='tanh')))

    model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
    model.add(Dense(5, activation = 'sigmoid', kernel_regularizer='l2'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [133]:
model = create_baseline()

In [134]:
model.fit(X_train, Y_train, epochs = 1, verbose = 1)

Epoch 1/10
15960/15960 [==============================] - 37s 2ms/step - loss: 0.4920 - acc: 0.9910
Epoch 2/10
15960/15960 [==============================] - 36s 2ms/step - loss: 0.1573 - acc: 0.9939
Epoch 3/10
15960/15960 [==============================] - 35s 2ms/step - loss: 0.1581 - acc: 0.9955
Epoch 4/10
15960/15960 [==============================] - 35s 2ms/step - loss: 0.1376 - acc: 0.9957
Epoch 5/10
15960/15960 [==============================] - 35s 2ms/step - loss: 0.1158 - acc: 0.9982
Epoch 6/10
15960/15960 [==============================] - 34s 2ms/step - loss: 0.1023 - acc: 0.9984
Epoch 7/10
15960/15960 [==============================] - 35s 2ms/step - loss: 0.1297 - acc: 0.9957
Epoch 8/10
15960/15960 [==============================] - 35s 2ms/step - loss: 0.1043 - acc: 0.9986
Epoch 9/10
15960/15960 [==============================] - 35s 2ms/step - loss: 0.1333 - acc: 0.9944
Epoch 10/10
15960/15960 [==============================] - 35s 2ms/step - loss: 0.1101 - acc: 0.9966

In [135]:

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_27 (Bidirectio (None, 128)               43008     
_________________________________________________________________
dense_29 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_30 (Dense)             (None, 5)                 645       
Total params: 60,165
Trainable params: 60,165
Non-trainable params: 0
_________________________________________________________________


In [136]:
model.predict(X_test)

array([[3.4114751e-03, 9.5469074e-04, 2.5558493e-05, 1.5450889e-01,
        8.0616621e-04],
       [3.0349051e-03, 1.1168236e-03, 2.3575652e-05, 1.7142487e-01,
        7.2144798e-04],
       [7.5324840e-04, 3.6452716e-04, 7.7957113e-04, 4.3224584e-04,
        2.5214130e-01],
       ...,
       [1.3372233e-03, 1.1960189e-03, 3.3721572e-01, 1.4136158e-05,
        6.5652956e-04],
       [2.9087658e-03, 1.1786586e-03, 2.1938673e-05, 1.8439685e-01,
        6.7242235e-04],
       [2.8504917e-01, 2.5371115e-05, 1.1337256e-03, 1.2432591e-03,
        6.7900470e-04]], dtype=float32)

In [137]:
print(Y_test)

[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]


In [138]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 99.85%


In [ ]:
# from preprocess_dataset_full import standardised_X as X, dummy_Y as Y